---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [294]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df[260]

'.Iredell Memorial Hospital, February 2000, after suicide attempt by cutting.Hx of Outpatient Treatment: Yes\n'

In [321]:
import re
def date_sorter():
    reSent1 = r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})' # mm/dd/yy, fillout if len(yy)==2
    reSent2 = r'(?P<month>Jan|Feb|Mar|Apr|May|June|Jul|Aug|Sep|Oct|Nov|Dec).?[a-z., -]+(?P<day>\d?\d)[a-z ,./-]+(?P<year>\d{2,4})' # Mar. 20 2020
    reSent3 = r'(?P<day>\d?\d)[ /-]+(?P<month>Jan|Feb|Mar|Apr|May|June|Jul|Aug|Sep|Oct|Nov|Dec)[a-z., /-]+(?P<year>\d{2,4})' # 20 Mar. 2020
    reSent4 = r'(?P<month>Jan|Feb|Mar|Apr|May|June|Jul|Aug|Sep|Oct|Nov|Dec)[a-z ]+(?P<year>\d{4})' # Mar 2020
    reSent5 = r'(?P<month>\d{1,2})/(?P<year>\d{4})' # 12/2020
    reSent6 = r'(?P<year>\d{4})' # (Jan 1) 2020

    reFrame1 = df.str.extract(reSent1)
    reFrame2 = df.str.extract(reSent2).dropna(how='all')
    reFrame2['month'] = reFrame2['month'].replace({r'Jan[a-z]*':'01', 
                                   r'Feb[a-z]*':'02',
                                   r'Mar[a-z]*':'03',
                                   r'Apr[a-z]*':'04',
                                   r'May[a-z]*':'05',
                                   r'Jun[a-z]*':'06',
                                   r'Jul[a-z]*':'07',
                                   r'Aug[a-z]*':'08',
                                   r'Sep[a-z]*':'09',
                                   r'Oct[a-z]*':'10',
                                   r'Nov[a-z]*':'11',
                                   r'Dec[a-z]*':'12'}, regex=True)
    reFrame3 = df.str.extract(reSent3).dropna(how='all')
    reFrame3['month'] = reFrame3['month'].replace({r'Jan[a-z]*':'01', 
                                   r'Feb[a-z]*':'02',
                                   r'Mar[a-z]*':'03',
                                   r'Apr[a-z]*':'04',
                                   r'May[a-z]*':'05',
                                   r'Jun[a-z]*':'06',
                                   r'Jul[a-z]*':'07',
                                   r'Aug[a-z]*':'08',
                                   r'Sep[a-z]*':'09',
                                   r'Oct[a-z]*':'10',
                                   r'Nov[a-z]*':'11',
                                   r'Dec[a-z]*':'12'}, regex=True)
    reFrame4 = df.str.extract(reSent4).dropna(how='all')
    reFrame4['month'] = reFrame4['month'].replace({r'Jan[a-z]*':'01', 
                                   r'Feb[a-z]*':'02',
                                   r'Mar[a-z]*':'03',
                                   r'Apr[a-z]*':'04',
                                   r'May[a-z]*':'05',
                                   r'Jun[a-z]*':'06',
                                   r'Jul[a-z]*':'07',
                                   r'Aug[a-z]*':'08',
                                   r'Sep[a-z]*':'09',
                                   r'Oct[a-z]*':'10',
                                   r'Nov[a-z]*':'11',
                                   r'Dec[a-z]*':'12'}, regex=True)
    reFrame5 = df.str.extract(reSent5).dropna(how='all')
    reFrame5.insert(1, 'day', '1')
    reFrame6 = df.str.extract(reSent6).dropna(how='all').to_frame(name='year')
    reFrame6.insert(0, 'day', '1')
    reFrame6.insert(0, 'month', '1')
#     reFrame5 = reFrame5.fillna(reFrame6)
    reFrame1 = reFrame1.fillna(reFrame2)
    reFrame1 = reFrame1.fillna(reFrame3)
    reFrame1 = reFrame1.fillna(reFrame4)
    reFrame1 = reFrame1.fillna(reFrame5)
    reFrame1 = reFrame1.fillna(reFrame6)
    reFrame1['year']=reFrame1['year'].str.replace(r'(^\d{2}\b)',lambda x: '19'+x.groups()[0][:2])
    reFrame1['month']= reFrame1['month'].astype(int)
    reFrame1['day']= reFrame1['day'].astype(int)
    reFrame1['year']= reFrame1['year'].astype(int) 
    sortedIdx = reFrame1.sort_values(by = ['year', 'month', 'day'], ascending=True).index
    sortedIdx = pd.Series(sortedIdx)
    return sortedIdx

In [322]:
date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: currently extract(expand=None) means expand=False (retu

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64